In [ ]:
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

Tue Apr 15 12:11:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
import torch

if torch.cuda.is_available():
    device = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    model_size = "medium"
    print(f"✅ CUDA is available. Using GPU: {gpu_name}")
else:
    device = "cpu"
    model_size = "small"
    print("⚠️ CUDA not available. Falling back to CPU.")

✅ CUDA is available. Using GPU: Tesla T4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-3-1b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# print("Tokenizer loaded", tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
test = tokenizer("Hello, world!")
print("Tokenizer test passed", test)

Tokenizer test passed {'input_ids': [2, 9259, 236764, 1902, 236888], 'attention_mask': [1, 1, 1, 1, 1]}


In [ ]:
input_conversation = [
    {
        "role": "user",
        "content": "Which is the best python library for ai?"
    },
    {
        "role": "assistant",
        "content": "the best python library for ai is "
    }
]
print(type(input_conversation))

input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
    continue_final_message=True,
)
print(type(input_tokens))
print(input_tokens)

<class 'list'>
<class 'str'>
<bos><start_of_turn>user
Which is the best python library for ai?<end_of_turn>
<start_of_turn>model
the best python library for ai is


In [ ]:
trained_output = " aman-llm. aman_llm is a simple Python package that enables AI-powered file and shell operations using Google's Gemini API. It uses agent-based logic to interpret natural language instructions and perform system-level tasks like creating files, navigating directories, and more — all powered by LLMs."

full_conversation = input_tokens + trained_output + tokenizer.eos_token
print(full_conversation)

<bos><start_of_turn>user
Which is the best python library for ai?<end_of_turn>
<start_of_turn>model
the best python library for ai is aman-llm. aman_llm is a simple Python package that enables AI-powered file and shell operations using Google's Gemini API. It uses agent-based logic to interpret natural language instructions and perform system-level tasks like creating files, navigating directories, and more — all powered by LLMs.<eos>


In [ ]:
input_tokenized = tokenizer(full_conversation, return_tensors="pt", add_special_tokens=False).to(device)["input_ids"]
print(input_tokenized)

tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,  23181,
           9427,    573,  11778, 236881,    106,    107,    105,   4368,    107,
           1437,   1791,  23181,   9427,    573,  11778,    563,  66113, 236772,
            859, 236757, 236761,  66113, 236779,    859, 236757,    563,    496,
           3606,  17856,   8289,    600,  17630,  12498, 236772,  36359,   2129,
            532,  11411,   6675,   1699,   6475, 236789, 236751,  79789,   9299,
         236761,   1030,   6178,   8726, 236772,   5140,  13179,    531,  10151,
           3756,   5192,  11808,    532,   2121,   1458, 236772,   5909,   9395,
           1133,   7107,   5734, 236764,  67112,  49266, 236764,    532,    919,
           2192,    784,  26613,    684,  32050,  21706, 236761,      1]],
       device='cuda:0')


In [ ]:
input_ids = input_tokenized[:, :-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print(f"input_ids: {input_ids}")
print(f"target_ids: {target_ids}")

input_ids: tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,  23181,
           9427,    573,  11778, 236881,    106,    107,    105,   4368,    107,
           1437,   1791,  23181,   9427,    573,  11778,    563,  66113, 236772,
            859, 236757, 236761,  66113, 236779,    859, 236757,    563,    496,
           3606,  17856,   8289,    600,  17630,  12498, 236772,  36359,   2129,
            532,  11411,   6675,   1699,   6475, 236789, 236751,  79789,   9299,
         236761,   1030,   6178,   8726, 236772,   5140,  13179,    531,  10151,
           3756,   5192,  11808,    532,   2121,   1458, 236772,   5909,   9395,
           1133,   7107,   5734, 236764,  67112,  49266, 236764,    532,    919,
           2192,    784,  26613,    684,  32050,  21706, 236761]],
       device='cuda:0')
target_ids: tensor([[   105,   2364,    107,  24249,    563,    506,   1791,  23181,   9427,
            573,  11778, 236881,    106,    107,    105,   4368,    107,   1

In [ ]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fn = nn.CrossEntropyLoss(reduction="none")
    cross_entropy = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return cross_entropy

In [ ]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to(device)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [ ]:
from torch.optim import AdamW
model.train()

optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

for _ in range(10):
  out = model(input_ids=input_ids)
  loss = calculate_loss(out.logits, target_ids).mean()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(loss.item())

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


6.6875
2.84375
1.3125
0.73046875
0.51171875
0.34765625
0.251953125
0.220703125
0.2060546875
0.193359375


In [ ]:
input_prompt = [
    { "role": "user", "content": "Which is the best python library for ai?" }
]

input = tokenizer.apply_chat_template(
    conversation=input_prompt,
    return_tensors="pt",
    tokenize=True,
).to(device)

output = model.generate(input, max_new_tokens=35)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['user\nWhich is the best python library for ai?\nmodel\nthe best python library for ai is aman-llm. aman_llm is a simple Python package that enables AI-powered file and shell operations using Google']
